In [2]:
import io
import pathlib
import urllib.request
import zipfile

import numpy as np
import pandas as pd

# Background (links good as of 2019-12-10)

<h2 style="color: red;">This takes a fair amount of RAM and time</h2>

This script fetches ~50 .sf1 state and territory zip files from the census server at the following location:

* https://www2.census.gov/census_2010/04-Summary_File_1/

These each of these zip files contains (among other things) a geographic file and a population file containing.

The fixed-width geo file is useful because it ties a  "Logical Record" to ZCTA codes. The CSV population file is useful because it contains the actual data we need for the logical records.

Specifically, we are pulling our data from population table 9 ("P9" ... see Summary File 1 documentation page  184).

    P5. HISPANIC OR LATINO ORIGIN BY RACE Universe: Total population (17)

The data for this table is as follows:

    P5 data:
        Total:
            Not Hispanic or Latino:
                White alone
                Black or African American alone
                American Indian and Alaska Native alone
                Asian alone
                Native Hawaiian and Other Pacific Islander alone 
                Some Other Race alone
                Two or More Races
            Hispanic or Latino:
                White alone
                Black or African American alone 
                American Indian and Alaska Native alone
                Asian alone
                Native Hawaiian and Other Pacific Islander alone 
                Some Other Race alone
                Two or More Races

## Notes:
* Geo table must be filtered by summary level == 871 (State-5-Digit ZIP Code Tabulation Area) so that it is limited to ZCTA records.
* The cut down geo table is joined to the population table using the LOGRECNO (Log Record Number)
* All Hispanic groups are condense to a single group to make consistent with surname data
* Asian and API are combined to make consistent with surname data
* "Some Other Race" is apporitoned abong the groups.

For more details, the technical documentation for Summary File 1 can be found here:
* https://www.census.gov/prod/cen2010/doc/sf1.pdf

In [3]:
# These are the start/stop indices for the fixed with geo file.
# It allows them to be easily converted to dataframes.
GEO_MAP_2010 = {
    'FILEID'  : (1  , 7  ),
    'STUSAB'  : (7  , 9  ),
    'SUMLEV'  : (9  , 12 ),
    'GEOCOMP' : (12 , 14 ),
    'CHARITER': (14 , 17 ),
    'CIFSN'   : (17 , 19 ),
    'LOGRECNO': (19 , 26 ),
    'REGION'  : (26 , 27 ),
    'DIVISION': (27 , 28 ),
    'STATE'   : (28 , 30 ),
    'COUNTY'  : (30 , 33 ),
    'COUNTYCC': (33 , 35 ),
    'COUNTYSC': (35 , 37 ),
    'COUSUB'  : (37 , 42 ),
    'COUSUBCC': (42 , 44 ),
    'COUSUBSC': (44 , 46 ),
    'PLACE'   : (46 , 51 ),
    'PLACECC' : (51 , 53 ),
    'PLACESC' : (53 , 55 ),
    'TRACT'   : (55 , 61 ),
    'BLKGRP'  : (61 , 62 ),
    'BLOCK'   : (62 , 66 ),
    'IUC'     : (66 , 68 ),
    'CONCIT'  : (68 , 73 ),
    'CONCITCC': (73 , 75 ),
    'CONCITSC': (75 , 77 ),
    'AIANHH'  : (77 , 81 ),
    'AIANHHFP': (81 , 86 ),
    'AIANHHCC': (86 , 88 ),
    'AIHHTLI' : (88 , 89 ),
    'AITSCE'  : (89 , 92 ),
    'AITS'    : (92 , 97 ),
    'AITSCC'  : (97 , 99 ),
    'TTRACT'  : (99 , 105),
    'TBLKGRP' : (105, 106),
    'ANRC'    : (106, 111),
    'ANRCCC'  : (111, 113),
    'CBSA'    : (113, 118),
    'CBSASC'  : (118, 120),
    'METDIV'  : (120, 125),
    'CSA'     : (125, 128),
    'NECTA'   : (128, 133),
    'NECTASC' : (133, 135),
    'NECTADIV': (135, 140),
    'CNECTA'  : (140, 143),
    'CBSAPCI' : (143, 144),
    'NECTAPCI': (144, 145),
    'UA'      : (145, 150),
    'UASC'    : (150, 152),
    'UATYPE'  : (152, 153),
    'UR'      : (153, 154),
    'CD'      : (154, 156),
    'SLDU'    : (156, 159),
    'SLDL'    : (159, 162),
    'VTD'     : (162, 168),
    'VTDI'    : (168, 169),
    'RESERVE2': (169, 172),
    'ZCTA5'   : (172, 177),
    'SUBMCD'  : (177, 182),
    'SUBMCDCC': (182, 184),
    'SDELM'   : (184, 189),
    'SDSEC'   : (189, 194),
    'SDUNI'   : (194, 199),
    'AREALAND': (119, 213),
    'AREAWATR': (213, 227),
    'NAME'    : (227, 317),
    'FUNCSTAT': (317, 318),
    'GCUNI'   : (318, 319),
    'POP100'  : (319, 328),
    'HU100'   : (328, 337),
    'INTPTLAT': (337, 348),
    'INTPTLON': (348, 360),
    'LSADC'   : (360, 362),
    'PARTFLAG': (362, 363),
    'RESERVE3': (363, 369),
    'UGA'     : (369, 374),
    'STATENS' : (374, 382),
    'COUNTYNS': (382, 390),
    'COUSUBNS': (390, 398),
    'PLACENS' : (398, 406),
    'CONCITNS': (406, 414),
    'AIANHHNS': (414, 422),
    'AITSNS'  : (422, 430),
    'ANRCNS'  : (430, 438),
    'SUBMCDNS': (438, 446),
    'CD113'   : (446, 448),
    'CD114'   : (448, 450),
    'CD115'   : (450, 452),
    'SLDU2'   : (452, 455),
    'SLDU3'   : (455, 458),
    'SLDU4'   : (458, 461),
    'SLDL2'   : (461, 464),
    'SLDL3'   : (464, 467),
    'SLDL4'   : (467, 470),
    'AIANHHSC': (470, 472),
    'CSASC'   : (472, 476),
    'CNECTASC': (474, 477),
    'MEMI'    : (476, 478),
    'NMEMI'   : (477, 478),
    'PUMA'    : (478, 483),
    'RESERVED': (483, 501),
}

# Our zip downloads have URLs that can be recreated with state/arrevs.
STATES = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'DC': 'District_of_Columbia',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New_Hampshire',
    'NJ': 'New_Jersey',
    'NM': 'New_Mexico',
    'NY': 'New_York',
    'NC': 'North_Carolina',
    'ND': 'North_Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'PR': 'Puerto_Rico',
    'RI': 'Rhode_Island',
    'SC': 'South_Carolina',
    'SD': 'South_Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West_Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming',    
}

# This is the template for the URLs
URL_TEMPLATE_ZIP = 'https://www2.census.gov/census_2010/04-Summary_File_1/{state}/{state_abbrev}2010.sf1.zip'

# This creates a URL with each and every state in the STATES dictioanry
urls = [
    URL_TEMPLATE_ZIP.format(state_abbrev=code.lower(), state=name)
    for code, name
    in STATES.items()
]

In [4]:
def dl_file(url):
    '''Helper func: downloads zip from URL and stores it in file-like object'''
    # Open request
    with urllib.request.urlopen(url) as f:
        # Read into bytes IO and return
        data = io.BytesIO(f.read())
        return data

def make_geo_df(data):
    '''Helper func: takes zip and creates a geographic file from data'''
    with zipfile.ZipFile(data) as zf:
        # Filter out everything except the ZipInfo (geo) for csv we want
        target = zf.filelist[0]
        # Read that CSV into BytesIO object
        geo_data = io.BytesIO(zf.read(target))
        # Read fixed-width file into dataframe
        geo_df = pd.read_fwf(
            geo_data, 
            header=None,
            # Use the GEO MAP but subtract from column start/stop
            colspecs=[
                (tuple_[0] - 1, tuple_[1] - 1)
                for tuple_
                in GEO_MAP_2010.values()
            ],
            dtype=str
        )
    # Give names to columns
    geo_df.columns = tuple(GEO_MAP_2010.keys())
    # Filter out all records that are not related to ZCTAs only
    # e.g. get rid of census block data
    geo_df = geo_df.loc[geo_df.SUMLEV == '871']
    # Keep the STUSAB (state), LOGRECONO (join key), and ZCTA5 (zip code proxy)
    geo_df = geo_df[['STUSAB', 'LOGRECNO', 'ZCTA5']]#.dropna(subset=['ZCTA5'])
    return geo_df

def make_pop_df(data):
    '''Helper func: Takes a zip and creates population df'''
    with zipfile.ZipFile(data) as zf:
        # Filter out everything except the ZipInfo for csv we want
        # This contains Table P5
        target = zf.filelist[3]
        # Read that CSV into BytesIO object
        pop_data = io.BytesIO(zf.read(target))
        pop_df = pd.read_csv(
            pop_data, 
            header=None,
            dtype=str
        )
        # Keep only a subset of columns and renames them
        pop_df = pop_df[[1, 4, 18, 19, 20, 21, 22, 23, 24, 25]]
        pop_df.columns = [
            'STUSAB',
            'LOGRECNO',
            'white',
            'black',
            'native',
            'asian',
            'pi',
            'other',
            'multiple',
            'hispanic',
        ]
        return pop_df

def merge_frames(geo_df, pop_df):
    '''Merges our GEO and POP frames'''
    # Merges common STUSAB and LOGRECNO fields
    merged = geo_df.merge(pop_df)
    # Set index to ZCTA5 and sort
    merged = merged.set_index('ZCTA5')
    merged = merged.sort_index()
    return merged
    
def create_df(url):
    '''Main function to download, join, and clean data for single state'''
    print('.', end='')
    # Download
    data   = dl_file(url)
    # Make dfs
    geo_df = make_geo_df(data)
    pop_df = make_pop_df(data)
    # Join DFs, sort, trip, and process
    df     = merge_frames(geo_df, pop_df)
    df = df.iloc[:, 2:]
    df = df.astype(np.float64)
    return df

# Create a dataframe for each URL and store in list
data = [
    create_df(url)
    for url
    in urls
]

print('\nDownload complete.')

..Download complete.


In [6]:
# Join all data into single dataframe and sort index
df = pd.concat(data)
df = df.sort_index()
df.head()

,white,black,native,asian,pi,other,multiple,hispanic
ZCTA5,,,,,,,,
30165,66.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
31905,2.0,1.0,1.0,0.0,1.0,0.0,0.0,4.0
35004,8947.0,931.0,28.0,167.0,0.0,10.0,130.0,214.0
35005,4655.0,2986.0,38.0,16.0,1.0,8.0,81.0,157.0
35006,2868.0,167.0,23.0,1.0,0.0,2.0,31.0,29.0


In [8]:
# Store column totals
totals = df.sum(axis=1)
totals.head()

ZCTA5
30165       70.0
31905        9.0
35004    10427.0
35005     7942.0
35006     3121.0
dtype: float64

In [10]:
# Store some other race so it can be divvyed up among other groups
other = df['other']
other.head()

ZCTA5
30165     0.0
31905     0.0
35004    10.0
35005     8.0
35006     2.0
Name: other, dtype: float64

In [11]:
# Create Asian or Pacific Islander (this is what surname uses)
df['api'] = df['asian'] + df['pi']
df.head()

,white,black,native,asian,pi,other,multiple,hispanic,api
ZCTA5,,,,,,,,,
30165,66.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31905,2.0,1.0,1.0,0.0,1.0,0.0,0.0,4.0,1.0
35004,8947.0,931.0,28.0,167.0,0.0,10.0,130.0,214.0,167.0
35005,4655.0,2986.0,38.0,16.0,1.0,8.0,81.0,157.0,17.0
35006,2868.0,167.0,23.0,1.0,0.0,2.0,31.0,29.0,1.0


In [12]:
# Drop columns we will no longer use
df = df.drop(columns=['other', 'asian', 'pi'])
df.head()

,white,black,native,multiple,hispanic,api
ZCTA5,,,,,,
30165,66.0,4.0,0.0,0.0,0.0,0.0
31905,2.0,1.0,1.0,0.0,4.0,1.0
35004,8947.0,931.0,28.0,130.0,214.0,167.0
35005,4655.0,2986.0,38.0,81.0,157.0,17.0
35006,2868.0,167.0,23.0,31.0,29.0,1.0


In [13]:
# Now determine what percent of the row each items makes up.
percentages = df.divide(totals, axis='rows')
percentages.head()

,white,black,native,multiple,hispanic,api
ZCTA5,,,,,,
30165,0.942857,0.057143,0.000000,0.000000,0.000000,0.000000
31905,0.222222,0.111111,0.111111,0.000000,0.444444,0.111111
35004,0.858061,0.089287,0.002685,0.012468,0.020524,0.016016
35005,0.586124,0.375976,0.004785,0.010199,0.019768,0.002141
35006,0.918936,0.053508,0.007369,0.009933,0.009292,0.000320


In [14]:
# Split up 'other' into the remaining rows based on the percents above
apportioned_other = percentages.multiply(other, axis='rows')
apportioned_other.head()

,white,black,native,multiple,hispanic,api
ZCTA5,,,,,,
30165,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
31905,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
35004,8.580608,0.892874,0.026853,0.124676,0.205236,0.160161
35005,4.688995,3.007807,0.038278,0.081592,0.158147,0.017124
35006,1.837872,0.107017,0.014739,0.019865,0.018584,0.000641


In [15]:
# Impute 'other' to the remaining groups based on percentage makeup
# quasi Iterative proportortional fit / matrix rake over single dimension
df += apportioned_other
df.head()

,white,black,native,multiple,hispanic,api
ZCTA5,,,,,,
30165,66.000000,4.000000,0.000000,0.000000,0.000000,0.000000
31905,2.000000,1.000000,1.000000,0.000000,4.000000,1.000000
35004,8955.580608,931.892874,28.026853,130.124676,214.205236,167.160161
35005,4659.688995,2989.007807,38.038278,81.081592,157.158147,17.017124
35006,2869.837872,167.107017,23.014739,31.019865,29.018584,1.000641


# Write data to module as CSV

In [12]:
current_directory = pathlib.Path().cwd()
project_directory = current_directory.parents[0]
data_directory    = project_directory / 'surgeo' / 'data'
df_path           = data_directory / 'population_2010.csv'
df.to_csv(df_path)